In [1]:
from pyspark.sql import Row 
from pyspark.sql.types import *
import pandas as pdd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
data = spark.read.csv("/FileStore/tables/train.csv",header=True, inferSchema=True)
data.show(1)

+-------------------+--------+--------------+---------+----------+--------------+------------------+-----------------+----------------+
 Dates|Category| Descript|DayOfWeek|PdDistrict| Resolution| Address| X| Y|
+-------------------+--------+--------------+---------+----------+--------------+------------------+-----------------+----------------+
2015-05-13 23:53:00|WARRANTS|WARRANT ARREST|Wednesday| NORTHERN|ARREST, BOOKED|OAK ST / LAGUNA ST|-122.425891675136|37.7745985956747|
+-------------------+--------+--------------+---------+----------+--------------+------------------+-----------------+----------------+
only showing top 1 row

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
win = Window().orderBy(year(data.Dates))
df_imb = data.withColumn("idx", row_number().over(win))

In [4]:
data.count()

Out[16]: 878049

In [5]:
c=df_imb.select("idx","Category")

In [6]:
df_imbbb=c.groupBy("idx").pivot("Category").count().fillna(0)

In [7]:
df_col=df_imbbb.columns
df_col

Out[135]: ['idx',
 'ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS']

In [8]:
from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit , CrossValidator

In [9]:
from pyspark.sql.functions import *
#df2 = df_imb.select(year(data.Dates).alias("Year"),month(data.Dates).alias("Month"),weekofyear(data.Dates).alias('dt_week_no'),"Category",minute(df_imb.Dates).alias("Minute"),unix_timestamp(df_imb.Dates).alias('dt_int').cast("double"),hour(df_imb.Dates).alias("Hour"),"X","Y")
#df2.show(5)
#,"DayOfWeek",,month(data.Dates).alias("Month"),, weekofyear(data.Dates).alias('dt_week_no'),,year(data.Dates).alias("Year"),year(df_imb.Dates).alias("Year"),"Address","DayOfWeek","PdDistrict","Address","DayOfWeek",month(df_imb.Dates).alias("Month"),
df2 = df_imb.select(minute(df_imb.Dates).alias("Minute"),unix_timestamp(df_imb.Dates).alias('dt_int').cast("double"),hour(df_imb.Dates).alias("Hour"),"PdDistrict",df_imb.X.cast("double"), df_imb.Y.cast("double"),"Category")

In [10]:
encoding_var = [i[0] for i in df2.dtypes if (i[1]=='string') & (i[0]!='Category')]
print("encoding_var:",encoding_var)

num_var = [i[0] for i in df2.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='Category')]
print("num_var:",num_var)

encoding_var: ['PdDistrict']
num_var: ['Minute', 'dt_int', 'Hour', 'X', 'Y']

In [11]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import IndexToString

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]

onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]

label_indexes = StringIndexer(inputCol = 'Category', outputCol = 'label',handleInvalid = 'keep')
labels = label_indexes.fit(df2).labels

label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labels)

assembler = VectorAssembler(inputCols = num_var + ['OHE_' + c for c in encoding_var], outputCol = "features")

In [12]:
train_data, test_data = df2.randomSplit([.8,.2],seed=1234)

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder , CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.tuning import TrainValidationSplitModel
from pyspark.sql import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import  DecisionTreeClassifier


rf = RandomForestClassifier(labelCol="label", featuresCol="features")
pipe_rf = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes,rf,label_converter])

lr = LogisticRegression(featuresCol = 'features', labelCol = 'label')
pipe_lr = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes,lr,label_converter])

In [14]:
rf1 = RandomForestClassifier()

print(rf1.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto' (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features. Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. (default: 5)
maxMemoryInMB: Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size. (default: 256)
minInfoGain: Minimum information gain for a split to be considered at a tree node. (default: 0.0)
minInstancesPerNode: Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1. (default: 1)
numTrees: Number of trees to train (>= 1). (default: 20)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
seed: random seed. (default: -5387697053847413545)
subsamplingRate: Fraction of the training data used for learning each decision tree, in range (0, 1]. (default: 1.0)

In [15]:
lr1 = LogisticRegression()

print(lr1.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal with 1 for binomial regression, or the number oflasses for multinomial regression. (undefined)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
regParam: regularization parameter (>= 0). (default: 0.0)
standardization: whether to standardize the training features before fitting the model. (default: True)
threshold: Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
upperBoundsOnCoefficients: The upper bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
upperBoundsOnIntercepts: The upper bounds on intercepts if fitting under bound constrained optimization. The bound vector size must be equal with 1 for binomial regression, or the number of classes for multinomial regression. (undefined)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)

In [16]:
params_rf = ParamGridBuilder().addGrid(rf.cacheNodeIds, [True]).addGrid(rf.maxDepth, [5]).addGrid(rf.minInfoGain, [0.001]).addGrid(rf.minInstancesPerNode, [3]).addGrid(rf.numTrees, [100]).build()
crossval_rf = CrossValidator(estimator=pipe_rf,estimatorParamMaps=params_rf,evaluator=BinaryClassificationEvaluator(),numFolds=3)



params_lr = ParamGridBuilder().addGrid(lr.elasticNetParam, [0.0]).addGrid(lr.fitIntercept, [True]).addGrid(lr.maxIter, [100]).addGrid(lr.regParam,[0.3]).build()
crossval_lr = CrossValidator(estimator=pipe_lr,estimatorParamMaps=params_lr,evaluator=BinaryClassificationEvaluator(),numFolds=3)
                          
                   

In [17]:
mod_rf= crossval_rf.fit(train_data)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions_rf = mod_rf.transform(test_data)
evaluator_rf = BinaryClassificationEvaluator()
evaluator_rf.evaluate(predictions_rf)

Out[242]: 0.6727007098208073

In [19]:
from pyspark.ml.pipeline import PipelineModel

best_rf_model = mod_rf.bestModel
best_rf_model.write().overwrite().save("/to_path/sparkml/best_rf_model")

model_rf_1 = PipelineModel.load("/to_path/sparkml/best_rf_model")
model_rf_2 = TrainValidationSplitModel(model_rf_1)

best_model_rf=model_rf_2.transform(test_data)

best_evaluator_rf = BinaryClassificationEvaluator()
best_evaluator_rf.evaluate(best_model_rf)

Out[209]: 0.6727007098208078

In [20]:
mod_lr = crossval_lr.fit(train_data)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [21]:
predictions_lr = mod_lr.transform(test_data)
evaluator_lr = BinaryClassificationEvaluator()
evaluator_lr.evaluate(predictions_lr)

Out[234]: 0.6369163977974117

In [22]:
Test = spark.read.csv("/FileStore/tables/test.csv",header=True, inferSchema=True)
Test.show(5)

+---+-------------------+---------+----------+--------------------+-------------------+----------------+
 Id| Dates|DayOfWeek|PdDistrict| Address| X| Y|
+---+-------------------+---------+----------+--------------------+-------------------+----------------+
 0|2015-05-10 23:59:00| Sunday| BAYVIEW|2000 Block of THO...|-122.39958770418998|37.7350510103906|
 1|2015-05-10 23:51:00| Sunday| BAYVIEW| 3RD ST / REVERE AV| -122.391522893042|37.7324323864471|
 2|2015-05-10 23:50:00| Sunday| NORTHERN|2000 Block of GOU...| -122.426001954961|37.7922124386284|
 3|2015-05-10 23:45:00| Sunday| INGLESIDE|4700 Block of MIS...| -122.437393972517|37.7214120621391|
 4|2015-05-10 23:45:00| Sunday| INGLESIDE|4700 Block of MIS...| -122.437393972517|37.7214120621391|
+---+-------------------+---------+----------+--------------------+-------------------+----------------+
only showing top 5 rows

In [23]:
#year(Test.Dates).alias("Year"),Test.Dates).alias("Month")(month,"Address","DayOfWeek"
df3 = Test.select("Id",minute(Test.Dates).alias("Minute"),unix_timestamp(Test.Dates).alias('dt_int').cast("double"),hour(Test.Dates).alias("Hour"),"PdDistrict",Test.X.cast("double"), Test.Y.cast("double"))
df3.show(5)

+---+------+------------+----+----------+-------------------+----------------+
 Id|Minute| dt_int|Hour|PdDistrict| X| Y|
+---+------+------------+----+----------+-------------------+----------------+
 0| 59|1.43130234E9| 23| BAYVIEW|-122.39958770418998|37.7350510103906|
 1| 51|1.43130186E9| 23| BAYVIEW| -122.391522893042|37.7324323864471|
 2| 50| 1.4313018E9| 23| NORTHERN| -122.426001954961|37.7922124386284|
 3| 45| 1.4313015E9| 23| INGLESIDE| -122.437393972517|37.7214120621391|
 4| 45| 1.4313015E9| 23| INGLESIDE| -122.437393972517|37.7214120621391|
+---+------+------------+----+----------+-------------------+----------------+
only showing top 5 rows

In [24]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
win = Window().orderBy("Hour")
df4 = df3.withColumn("idx", row_number().over(win))
df4.show(2)

+---+------+------------+----+----------+-----------------+----------------+---+
 Id|Minute| dt_int|Hour|PdDistrict| X| Y|idx|
+---+------+------------+----+----------+-----------------+----------------+---+
313| 52|1.43121912E9| 0| NORTHERN|-122.428666208752|37.7757342027431| 1|
314| 44|1.43121864E9| 0| MISSION|-122.419385927086|37.7628330926795| 2|
+---+------+------------+----+----------+-----------------+----------------+---+
only showing top 2 rows

In [25]:
dff4=df4.select("Id","idx")
dff4.show(5)

+---+---+
 Id|idx|
+---+---+
313| 1|
314| 2|
315| 3|
316| 4|
317| 5|
+---+---+
only showing top 5 rows

In [26]:
df3=df3.drop("Id")

In [27]:
#Kaggle Submission

In [28]:
p=mod_rf.transform(df3)
p.show(1)

+------+------------+----+----------+-------------------+----------------+--------------------+--------------------+--------------------+----------+--------------+
Minute| dt_int|Hour|PdDistrict| X| Y| features| rawPrediction| probability|prediction|predictedLabel|
+------+------------+----+----------+-------------------+----------------+--------------------+--------------------+--------------------+----------+--------------+
 59|1.43130234E9| 23| BAYVIEW|-122.39958770418998|37.7350510103906|[59.0,1.43130234E...|[10.2510240597388...|[0.10251024059738...| 1.0|OTHER OFFENSES|
+------+------------+----+----------+-------------------+----------------+--------------------+--------------------+--------------------+----------+--------------+
only showing top 1 row

In [29]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
win = Window().orderBy("Hour")
df5 = p.withColumn("idx", row_number().over(win))
df5.show(2)

+------+------------+----+----------+-----------------+----------------+--------------------+--------------------+--------------------+----------+--------------+---+
Minute| dt_int|Hour|PdDistrict| X| Y| features| rawPrediction| probability|prediction|predictedLabel|idx|
+------+------------+----+----------+-----------------+----------------+--------------------+--------------------+--------------------+----------+--------------+---+
 52|1.43121912E9| 0| NORTHERN|-122.428666208752|37.7757342027431|[52.0,1.43121912E...|[14.2206422496083...|[0.14220642249608...| 1.0|OTHER OFFENSES| 1|
 44|1.43121864E9| 0| MISSION|-122.419385927086|37.7628330926795|[44.0,1.43121864E...|[12.2236377257285...|[0.12223637725728...| 1.0|OTHER OFFENSES| 2|
+------+------------+----+----------+-----------------+----------------+--------------------+--------------------+--------------------+----------+--------------+---+
only showing top 2 rows

In [30]:
df6=df5.select("idx","predictedLabel")
df6.show(2)

+---+--------------+
idx|predictedLabel|
+---+--------------+
 1|OTHER OFFENSES|
 2|OTHER OFFENSES|
+---+--------------+
only showing top 2 rows

In [31]:
df7=df6.groupBy("idx").pivot("predictedLabel").count().fillna(0)
df7.show(5)

+---+-------------+-------------+--------------+-------------+
idx|DRUG/NARCOTIC|LARCENY/THEFT|OTHER OFFENSES|VEHICLE THEFT|
+---+-------------+-------------+--------------+-------------+
 1| 0| 0| 1| 0|
 2| 0| 0| 1| 0|
 3| 0| 0| 1| 0|
 4| 0| 0| 1| 0|
 5| 0| 1| 0| 0|
+---+-------------+-------------+--------------+-------------+
only showing top 5 rows

In [32]:
unique_list=[]
for k in df_col:
  if k not in df7.columns:
    unique_list.append(k)       
            
unique_list

Out[193]: ['ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'WARRANTS',
 'WEAPON LAWS']

In [33]:
for i in unique_list:
  df7=df7.withColumn(i,lit(0))
df7.show(2)

+---+-------------+-------------+--------------+-------------+-----+-------+----------+-------+--------+------------------+---------------------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-----------+---------+--------------+------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+--------+-----------+
idx|DRUG/NARCOTIC|LARCENY/THEFT|OTHER OFFENSES|VEHICLE THEFT|ARSON|ASSAULT|BAD CHECKS|BRIBERY|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUNKENNESS|EMBEZZLEMENT|EXTORTION|FAMILY OFFENSES|FORGERY/COUNTERFEITING|FRAUD|GAMBLING|KIDNAPPING|LIQUOR LAWS|LOITERING|MISSING PERSON|NON-CRIMINAL|PORNOGRAPHY/OBSCENE MAT|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|SEX OFFENSES FORCIBLE|SEX OFFENSES NON FORCIBLE|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TREA|TRESPASS|VANDALISM|WARRANTS|WEAPON LAWS|
+---+-------------+-------------+--------------+-------------+-----+-------+----------+-------+--------+------------------+---------------------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-----------+---------+--------------+------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+--------+-----------+
 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 2| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-------------+-------------+--------------+-------------+-----+-------+----------+-------+--------+------------------+---------------------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-----------+---------+--------------+------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+--------+-----------+
only showing top 2 rows

In [34]:
df7=df7.select(*[df_col])

In [35]:
examp = dff4.join(df7,on ="idx")
final_dataset=examp.drop("idx")
final_dataset=final_dataset.orderBy("Id")


In [36]:
final_dataset.show(2)

+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 Id|ARSON|ASSAULT|BAD CHECKS|BRIBERY|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUG/NARCOTIC|DRUNKENNESS|EMBEZZLEMENT|EXTORTION|FAMILY OFFENSES|FORGERY/COUNTERFEITING|FRAUD|GAMBLING|KIDNAPPING|LARCENY/THEFT|LIQUOR LAWS|LOITERING|MISSING PERSON|NON-CRIMINAL|OTHER OFFENSES|PORNOGRAPHY/OBSCENE MAT|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|SEX OFFENSES FORCIBLE|SEX OFFENSES NON FORCIBLE|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TREA|TRESPASS|VANDALISM|VEHICLE THEFT|WARRANTS|WEAPON LAWS|
+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
only showing top 2 rows

In [37]:
print((final_dataset.count(), len(final_dataset.columns)))

(884262, 40)

In [38]:
final_dataset.where(col("Id")==882000).show()

+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 Id|ARSON|ASSAULT|BAD CHECKS|BRIBERY|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUG/NARCOTIC|DRUNKENNESS|EMBEZZLEMENT|EXTORTION|FAMILY OFFENSES|FORGERY/COUNTERFEITING|FRAUD|GAMBLING|KIDNAPPING|LARCENY/THEFT|LIQUOR LAWS|LOITERING|MISSING PERSON|NON-CRIMINAL|OTHER OFFENSES|PORNOGRAPHY/OBSCENE MAT|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|SEX OFFENSES FORCIBLE|SEX OFFENSES NON FORCIBLE|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TREA|TRESPASS|VANDALISM|VEHICLE THEFT|WARRANTS|WEAPON LAWS|
+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
882000| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+

In [39]:
dbutils.fs.rm("/FileStore/df",True)

Out[198]: True

In [40]:
final_dataset.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/final_dataset.csv")